In [1]:
import sys
sys.path.append("..")
from master import cds_utils as utils
# from master.cds_utils import *
import pandas as pd
import numpy as np
import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.preprocessing import MinMaxScaler
import os
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from keras import backend as K
from sklearn.metrics import roc_auc_score

/Users/cindy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/cindy/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
## The metric for the model

def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [3]:
# ## Preparing Dataset

# def preprocessing(dataframe_x,labels):
#     list_of_x=[]
#     list_of_y=np.zeros([1,1])

#     for i in dataframe_x:
#         list_of_x.append(np.append(0,i))
    
#     list_of_y.fill(labels[2])
    
#     # reshape input to be [samples, time steps, features]
#     list_of_x = np.reshape(list_of_x, (1, 10, 17))
    
#     return list_of_x,list_of_y

In [8]:
class The_LSTM_Model:
    
#     def __init__(self):
#         pass

    ## Preparing Dataset

    def preprocessing(self, dataframe_x,labels, size, features):
        list_of_x=[]
        list_of_y=np.zeros([1,1])
        
        for i in dataframe_x.values:
#             list_of_x.append(np.append(dat,i))
              list_of_x.append([i])
            
        list_of_y.fill(labels[2])

        # reshape input to be [samples, time steps, features]
        list_of_x = np.reshape(list_of_x, (1, size, features))

        return list_of_x,list_of_y

    ## Training and evaluating the model

    ## Train model in windows
    def train_model(self, X_train,y_train, size):
            # fix random seed for reproducibility
            np.random.seed(7)

            model.fit(X_train, y_train, epochs=3, batch_size=size)

    ## Evaluates model based on test set
    def evaluate_model(self, X_train,y_train,accuracy=0,iteration=0):

            # Final evaluation of the model
            scores = model.evaluate(X_train, y_train, verbose=0)

            #To get average accuracy of model
            accuracy+=scores[1]*100
            iteration+=1
            prediction=model.predict(X_train)

            return accuracy,iteration, prediction[0][0]


In [9]:
if __name__=='__main__':
    
    size=2
    spl_value=0.5
    
    lstm_model=The_LSTM_Model()
    # STEP 0: Check if combined dataset exists. If exists, GO TO STEP 4
    filename = '../master/GOOGL - main.csv'
    if os.path.isfile(filename):
        dataset = pd.read_csv(filename, parse_dates=[0])
        dataset.set_index('Date', inplace=True)
    else:
        # STEP 1:: Aggregate financial data from discrete csv files
        #compile_financial('AAPL/')

        # STEP 2: Extract csv files into pandas dataframe
        # (1) financial dataframe
        financial = pd.read_csv('../master/GOOGL - financial.csv', parse_dates=[1])
        financial = interpolate_data(financial, method='zero')
        # (2) price dataframe
        price = pd.read_csv('../master/GOOGL - price.csv', parse_dates=[0]) # path to price 
        price.set_index('Date', inplace=True)
        # (3) technical dataframe
        technical = pd.read_csv('../master/GOOGL - technical.csv', parse_dates=[0])
        technical.set_index('Date', inplace=True)
        # camel case column names for technical
        for old_column in technical:
            new_column = ' '.join([word.title() for word in old_column.split('_')])
            technical.rename(columns={old_column:new_column}, inplace=True)

        # STEP 3: Join different datasets based on overlapping dates
        dataset = combine_datasets(financial=financial, price=price, technical=technical)

    # STEP 4: Split the dataset into train and test
    train, test = utils.train_test_split(dataset, spl=spl_value)

    # STEP 5: Parse the train/test dataframe into a data generator
    data_gen = utils.data_generator(train, train_days=size, next='day')
    i=0
    ##100
    ## 200
    lstm_out=200
    model = Sequential()
    model.add(LSTM(lstm_out, dropout= 0.5,return_sequences=True))
    model.add(LSTM(200, dropout= 0.5))
#     list_of_x = np.reshape(list_of_x, (1, 2, 17))
    
#     model.add(LSTM(lstm_out, dropout= 0.5,return_sequences=True))
#     model.add(LSTM(200,dropout= 0.5, return_sequences=True))
#     model.add(LSTM(200))
#     model.add(LSTM(50))
    # each 60 outputs will have a dimension of 100
    model.add(Dense(1, activation='sigmoid'))
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=[auc])
   
    scaler = MinMaxScaler(feature_range=(0, 1))
    y_is_one=0
    y_is_zero=0
    
    for x_train,y_train in data_gen:
        
#         print(len(x_train))
#         print(x_train)
        
        features=len(x_train.columns)
        
        # x_train=x_train.drop("Quarter",axis=1)
#         x_train = scaler.fit_transform(x_train.values)
        if y_train[2]==1:
             y_is_one+=1
        else:
             y_is_zero+=1
            
        x,y=lstm_model.preprocessing(x_train, y_train,size, features)
        
        
        lstm_model.train_model(x,y,size)
#         i+=1
#         if i >10:
#             break
    
    print(y_is_one,y_is_zero)
    
            
    data_gen = utils.data_generator(test, train_days=size, next='day')
#     j=0
    accuracy=0
    iteration=0
    scaler = MinMaxScaler(feature_range=(0, 1))
    y_true=[]
    y_predict=[]
    
    for x_test,y_test in data_gen:
#         scaler = MinMaxScaler(feature_range=(0, 1))
#         x_test=x_test.drop("Quarter",axis=1)
#         x_test = scaler.fit_transform(x_test.values)

        features=len(x_test.columns)
        y_true.append(y_test[2])
        x,y=lstm_model.preprocessing(x_test, y_test,size, features)
        accuracy,iteration, prediction=lstm_model.evaluate_model(x,y,accuracy,iteration)
        
        y_predict.append(prediction)
#         if prediction<=0.5:
#             y_predict.append(0)
#         else:
#             y_predict.append(1)
        
#         j+=1
#         if j>10:
#             break
    print("Average")
    print(accuracy/iteration)
    
    the_score=roc_auc_score(y_true, y_predict)
    print(the_score)
    
#     summarize history for accuracy
#     plt.plot(history.history['acc'])
#     plt.plot(history.history['val_acc'])
#     plt.title('model accuracy')
#     plt.ylabel('accuracy')
#     plt.xlabel('epoch')
#     plt.legend(['train', 'test'], loc='upper left')
#     plt.show()
#     summarize history for loss
#     plt.plot(history.history['loss'])
#     plt.plot(history.history['val_loss'])
#     plt.title('model loss')
#     plt.ylabel('loss')
#     plt.xlabel('epoch')
#     plt.legend(['train', 'test'], loc='upper left')
#     plt.show()
    




Epoch 1/3
1/1 [==============================] - 3s 3s/step - loss: 0.6920 - auc: 1.0000
Epoch 2/3
1/1 [==============================] - 0s 13ms/step - loss: 0.6869 - auc: 1.0000
Epoch 3/3
1/1 [==============================] - 0s 12ms/step - loss: 0.6762 - auc: 1.0000
Epoch 1/3
1/1 [==============================] - 0s 11ms/step - loss: 0.6616 - auc: 1.0000
Epoch 2/3
1/1 [==============================] - 0s 12ms/step - loss: 0.6621 - auc: 1.0000
Epoch 3/3
1/1 [==============================] - 0s 12ms/step - loss: 0.6523 - auc: 1.0000
Epoch 1/3
1/1 [==============================] - 0s 12ms/step - loss: 0.6345 - auc: 1.0000
Epoch 2/3
1/1 [==============================] - 0s 12ms/step - loss: 0.6477 - auc: 1.0000
Epoch 3/3
1/1 [==============================] - 0s 13ms/step - loss: 0.5951 - auc: 1.0000


ValueError: cannot reshape array of size 33 into shape (1,2,11)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_true, y_predict).ravel()
print(tn,fp,fn,tp)